# k-means clustering (theory)
On the slide page 8-10 the k-means clustering problem was introduced. 

<b>Question 1 (a): </b>What is the objective of k-means clustering problem? 

<b>Question 1 (b): </b>Rewrite the objective for $\text{dist}(x,y)=||x-y||$

On slide 10-12 Lloyd's algorithm was introduced. Let $x_1,...,x_n\in \mathbb{R}^d$. The algorithm has two steps

- for i=1,...,T:
  - Update centroid: For $i=1,...,k$ let $\mu_i = \frac{1}{|C_i|} \sum_{x\in C_i}x$
  - Update clustering: Assign $x_i$ to cluster $C_j$ where $j=\arg \min ||x-\mu_j||^2$ for $i=1,..., n$


<b>Question 2: </b>Consider the image below. What are the centroids? Can you interpret the cost visually?

<img src="http://dovgalecs.com/blog/wp-content/uploads/2012/11/untitled.jpg" style="width: 200px; height:auto" />

<b>Question 3: </b>What is the running time of Lloyds algorithm? Let $T$ be the number of iterations, $n$ number of data points, $d$ dimension of the datapoints and $k$ the number of clusters. 

<b>Question 4: </b>Does Lloyd's algorithm provide any guarantees for finding the optimal solution?

<b>Question 5: </b>Can a cluster of a k-means clustering be empty? 

HINT: Must a cluster of a k-means clustering have at least one object?

<b>Question 6: </b>Where does Lloyds algorithm attempt to minimize the k-means objective?  

HINT: Try differentiate objective function and solve for equal zero.  

<b>Question 7: </b>Is Lloyd's algorithm guaranteed to converge?

HINT: How many different clusterings are there? Finite/Infinite? Could Lloyd's algorithm find the same clustering twice? Does the cost decrease/increase between iterations?

<b>Bonus Point: </b>
It is important that you realize there is a different between the k-means clustering problem and Lloyd's algorithm. Lloyd's algorithm is sometimes refered to as "the k-means algorithm" due to its extensive use. This might make it a bit confusing to distinguish between problem and algorithm. 

# k-means clustering (code)
In this exercise you must implement Lloyd's algorithm. To test our implementation we will need data. For this we will use the Iris dataset similar to the book [ZM]. The data set has three clases so we <i>cheat</i> by setting $k=3$

In [ ]:
# Load the Iris data set
import sklearn.datasets
iris = sklearn.datasets.load_iris()
X = iris['data'][:,0:2] # reduce dimensions so we can plot what happens.
k = 3
print(X.shape)

## Implementing Lloyds algorithm
The following implementation of Lloyd's algorithm is similar to the previous pseudo-code, except we added printing and stopping.

- for i=1,...,T:
  - Update centroid: For $i=1,...,k$ let $\mu_i = \frac{1}{|C_i|} \sum_{x\in C_i}x$
  - Update clustering: Assign $x_i$ to cluster $C_j$ where $j=\arg \min ||x-\mu_j||^2$
  - Compute and print cost of current clustering
  - If cost didn't improve from last iteration stop

Your job is to implement the two update operations. They are marked in the code as usual. One thing that might be a bit confusing is that clusterings are represented as a $n$-dimensional array `clustering`. If the point $x_i$ should be in cluster $j$ we have that `clustering[i]=j`.  

In [ ]:
import numpy as np

def lloyds_algorithm(X, k, T):
    """ Clusters the data of X into k clusters using T iterations of Lloyd's algorithm. 
    
        Parameters
        ----------
        X : Data matrix of shape (n, d)
        k : Number of clusters.
        T : Maximum number of iterations to run Lloyd's algorithm. 
        
        Returns
        -------
        clustering: A vector of shape (n, ) where the i'th entry holds the cluster of X[i].
        centroids:  The centroids/average points of each cluster. 
        cost:       The cost of the clustering 
    """
    n, d = X.shape
    
    # Initialize clusters random. 
    clustering = np.random.randint(0, k, (n, )) 
    centroids  = np.zeros((k, d))
    
    # Used to stop if cost isn't improving (decreasing)
    cost = 0
    oldcost = 0
    
    # Column names
    print("Iterations\tCost")
    
    for i in range(T):
        
        # Update centroid
        
        # YOUR CODE HERE
        # END CODE

        
        # Update clustering 
        
        # YOUR CODE HERE
        # END CODE
        
        
        # Compute and print cost
        cost = 0
        for j in range(n):
            cost += np.linalg.norm(X[j] - centroids[clustering[j]])**2    
        print(i+1, "\t\t", cost)
        
        
        # Stop if cost didn't improve more than epislon (decrease)
        if np.isclose(cost, oldcost): break #TODO
        oldcost = cost
        
    return clustering, centroids, cost

clustering, centroids, cost = lloyds_algorithm(X, 3, 100)

The random initialization sometimes causes the algorithm to get stuck at different local minimum. This causes different runs to get different scores. In practice this is usually handled by running the algorithm several times and picking the best run. 

The following code runs Lloyd's algorithm 50 times and plots the score of each run. Because the data set is fairly small, $n=150$, most of the runs will get the same score. 

In [ ]:
%matplotlib notebook 
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(8,3))    
costs = []

for i in range(100):
    _, _, cost = lloyds_algorithm(X, 3, 100)
    costs.append(cost)
    ax.plot(costs, 'bx')
    fig.canvas.draw()

To check your implementation you should run <a href="http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html">sklearns</a> implementation of Lloyds algorithm. By default the implementation repeats the algorithm $10$ times and picks the best result. A sanity check for your implementation of Lloyd's algorithm is to check that the scores are roughly the same. 

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
print(-kmeans.score(X))

To get a visual understanding of the algorithm, the following code visualizes each step of the algorithm. Just copy and paste the answer from your implementation from above and run.

In [ ]:
%matplotlib notebook 
import matplotlib.pyplot as plt
import time

def lloyds_algorithm_visualize(X, k, T):
    """ Clusters the data of X into k clusters using T iterations of Lloyd's algorithm. 
        The data is assumed to have dimension 2 and each step of the algorithm is visualized. 
    
        Parameters
        ----------
        X : Data matrix of shape (n, d)
        k : Number of clusters.
        T : Maximum number of iterations to run Lloyd's algorithm. 
        
        Returns
        -------
        clustering: A vector of shape (n, ) where the i'th entry holds the cluster of X[i]. 
    """
    n, d = X.shape
    
    assert d == 2, "The data is assumed to have dimension 2 so we can visualize it. "
    
    # Initialize clusters random. 
    clustering = np.random.randint(0, k, (n, )) 
    centroids  = np.zeros((k, d))
    
    # Used to stop if cost isn't improving (decreasing)
    cost = 0
    oldcost = 0
    
    # Initialize subplot for visualization
    fig, ax = plt.subplots(1, 1, figsize=(9,4)) 
    ax.axis('off')
    colors = ["r", "g", "b"]
    
    # Column names
    print("Iteration\tCost")
    
    for i in range(T):
        
        # Update centroid
        
        # YOUR CODE HERE
        # END CODE

        
        # Update clustering 
        
        # YOUR CODE HERE
        # END CODE
        
        
        # Draw clusters
        ax.cla()
        for j in range(k):
            centroid = centroids[j]
            c = colors[j]
            ax.scatter(centroid[0], centroid[1], s=123, c=c, marker='^')
            data = X[clustering==j]
            x = data[:,0]
            y = data[:,1]
            ax.scatter(x, y, s=3, c=c)
            
        fig.canvas.draw()
        time.sleep(1)
        
        # Compute and print cost
        cost = 0
        for j in range(n):
            cost += np.linalg.norm(X[j] - centroids[clustering[j]])**2    
        print(i+1, "\t\t", cost)
        
        
        # Stop if cost didn't improve (decrease)
        if np.isclose(cost, oldcost): break #TODO
        oldcost = cost
        
    return clustering, centroids, cost

clustering, centroids, cost = lloyds_algorithm_visualize(X, 3, 100)

## Evaluating the clustering using silhuette coefficient
In the lecture Ira talked about how one can compare different clusters.

In [ ]:
def silhouette(data, clustering): # give figure at TA session
    n, d = data.shape

    # YOUR CODE HERE
    s=...
    # END CODE

    return s

silhouette(X, clustering)

## Using clustering for image compression data
In this exercise you must use Lloyd's algorithm for image compression. The following code downloads and displays the two images we will consider: 

In [ ]:
import scipy.misc

def download_image(url):
    filename = url[url.rindex('/')+1:]
    try:
        with open(filename, 'rb') as fp:
            return scipy.misc.imread(fp) / 255
    except FileNotFoundError:
        import urllib.request
        with open(filename, 'w+b') as fp, urllib.request.urlopen(url) as r:
            fp.write(r.read())
            return scipy.misc.imread(fp) / 255
 
img_facade = download_image('https://users-cs.au.dk/rav/ml/handins/h4/nygaard_facade.jpg')
img_stairs = download_image('https://users-cs.au.dk/rav/ml/handins/h4/nygaard_stairs.jpg')

fig, ax = plt.subplots(1, 2, figsize=(9, 3))
ax[0].imshow(img_facade)
ax[1].imshow(img_stairs)
plt.show()

Each pixel of the above images are represented by three values: (R, G, B). By using clustering we can find groups of pixels that are similar, and represent each group of pixels just by its centroid. The code below implements this idea. Try run it (it might take some time to run)

In [ ]:
def compress_kmeans(im, k, T):
    height, width, depth = im.shape
    data = im.reshape((height * width, depth))
    clustering, centroids, score = lloyds_algorithm(data, k, 5)
    
    # make each entry of data to the value of it's cluster
    data_compressed = data
    
    for i in range(k): data_compressed[clustering == i] = centroids[i] 
    
    im_compressed = data_compressed.reshape((height, width, depth))
    plt.figure()
    plt.imshow(im_compressed)
    plt.show()

T = 10
k = 4
compress_kmeans(img_facade, k, T)

<b>Question 1</b>: Do you think higher/lower values of $k$ and $T$ gives "better" images? Try experiment with different values and make the best image you can. 

<b>Bonus Question: </b>(This exercise is not important for understanding the theory of the course) <br>
Try to modify the code to draw the image in each iteration. This should allow you to see how better a better clustering corresponds to selecting more representative colors. You can find inspiration to interactive plotting in the function `lloyds_algorithm_visualize` above. 

# Clustering digits
In previous weeks we did supervised learning on images of digits. In this exercise we will perform clustering on digits. Remember clustering can be considered a type of unsupervised learning. The main difference to what we did before is that  will attempt to find patterns in the data without using the labels.  

You can use the AUDigits if you want. The following code uses a data set of images called MNIST. They are almost identical. The only reason for using MNIST is that we can import it with just two lines of code. 

In [ ]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/")

X = mnist.train.images
y = mnist.train.labels

print(X.shape)

The following code runs Lloyd's algorithm on 5000 images from the MNIST dataset. It then visualizes the found centroids. 

In [ ]:
# One cluster for each digit
k = 10

# Run Lloyd's algorithm on 5000 images from the MNIST dataset. 
clustering, centroids, score = lloyds_algorithm(X[:1000], 10, 50)
print(centroids.shape)
fig, ax = plt.subplots(1, k, figsize=(8, 1))

for i in range(k):
    ax[i].imshow(centroids[i].reshape(28, 28), cmap='gray')
    
plt.show()

<b>Question 1: </b>Why do the centroids look like images of digits?

<b>Question 2: </b>Is it possible that not all digits are present? Why could this happen?